In [14]:
from src import environments
import nest_asyncio
nest_asyncio.apply()
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
import numpy as np
from src.environments import EnvTensorOne, EnvTensorSix
import tf_agents
from tf_agents.specs import tensor_spec
from tf_agents.policies import py_tf_eager_policy
from tf_agents.drivers import py_driver

from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy
import tensorflow as tf
from tf_agents.agents.ppo import ppo_agent
from tf_agents.networks import actor_distribution_network, value_network
from tf_agents.trajectories import trajectory
from tf_agents.trajectories import time_step as ts

In [2]:
tf.config.list_physical_devices('GPU')

2023-07-15 00:37:50.229853: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-15 00:37:50.246764: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-15 00:37:50.246891: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
reward= {
    "damageEnemyStatue": 5,
    "damageEnemyUnit": 4,
    "killEnemyStatue": 4,
    "killEnemyUnit": 2,
    "healFriendlyStatue": 1,
    "healTeammatel": 2,
    "healTeammate2": 2,
    "timeSpentHomeBase": 0,
    "timeSpentHomeTerritory": 0,
    "timeSpentAwayTerritory": 3,
    "timeSpentAwayBase": 0,
    "damageTaken": -1,
    "friendlyFire": -2,
    "healEnemy": -1,
    "fallDamageTaken": -3,
    "statueDamageTaken": -1,
    "manualBonus": 0,
    "victory": 0,
    "loss": 0,
    "tie": 0,
    "teamSpirit": 0,
    "timeScaling": 0,
}
args = {"home_team":[
    {
        'primaryColor': '#00FFFF',
        'slots': ['Blaster', 'ParalyzingDart', 'Pistol']
    },
    {
        'primaryColor': '#89CFF0',
        'slots': ['Blaster', 'ParalyzingDart', 'Pistol']
    },
    {
        'primaryColor': '#0000FF',
        'slots': ['Pistol', 'Blaster', 'ParalyzingDart']
    },
],
"away_team":[
            {
                'primaryColor': '#880808',
                'slots': ['Pistol', 'HealingGland', 'Blaster']
            },
            {
                'primaryColor': '#880808',
                'slots':
                ['Blaster', 'ParalyzingDart', 'Pistol']
            },
            {
                'primaryColor': '#880808',
                'slots': ['Pistol', 'HealingGland', 'Blaster']
            },
        ]}
env = EnvTensorOne(reward_function = reward, turbo_mode=True, **args)

In [5]:
# Define the network for the agent.
actor_net = actor_distribution_network.ActorDistributionNetwork(
    env.observation_spec(),
    env.action_spec(),
    fc_layer_params=(128, 100, 64))

value_net = value_network.ValueNetwork(
    env.observation_spec(),
    fc_layer_params=(128, 100, 64))

# Define the keras optimizer.
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

# Define the train step counter.
train_step_counter = tf.Variable(0)

# Create the PPO agent.
agent = ppo_agent.PPOAgent(
    env.time_step_spec(),
    env.action_spec(),
    optimizer,
    actor_net=actor_net,
    value_net=value_net,
    train_step_counter=train_step_counter)

# Initialize the agent.
agent.initialize()

2023-07-15 00:37:52.323142: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-15 00:37:52.323295: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-15 00:37:52.323384: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [6]:
from tf_agents.replay_buffers import tf_uniform_replay_buffer

replay_buffer_capacity = 1000
batch_size = 1
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=batch_size,
    max_length=replay_buffer_capacity)

In [7]:
import os
#current directory
current_dir = os.getcwd()
#save in checkpoints folder in the current directory
checkpoint_path = os.path.join(current_dir, "checkpoints")

agent_checkpointer = tf_agents.utils.common.Checkpointer(
    checkpoint_path,
    max_to_keep=1,
    agent = agent,
    policy = agent.policy,
    replay_buffer = replay_buffer,
    global_step = train_step_counter
)
tf.compat.v1.train.get_global_step()
agent_checkpointer.initialize_or_restore()
saver = tf_agents.policies.policy_saver.PolicySaver(agent.policy)

Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.


Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.


In [8]:
dataset = replay_buffer.as_dataset(
    sample_batch_size=32,
    num_steps=10,
    single_deterministic_pass=False).prefetch(3)

iterator = iter(dataset)

Instructions for updating:
Use `tf.data.Dataset.counter(...)` instead.


Instructions for updating:
Use `tf.data.Dataset.counter(...)` instead.


Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.
2023-07-15 00:37:53.398022: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


In [9]:
# For analysis
# env = EnvTensorOne(reward_function = reward, turbo_mode=False, **args)
# time_step = env.reset(policy=agent.policy)
# reward = 0

In [10]:
# while not time_step.is_last():
#     action_step = agent.collect_policy.action(time_step)
#     next_time_step = env.step(action_step.action)
#     reward = next_time_step._asdict()['reward'].numpy()
#     if reward != 0:
#         print(reward)
#     traj = trajectory.from_transition(time_step, action_step, next_time_step)
#     batch = tf.nest.map_structure(lambda t: tf.expand_dims(t, 0), traj)
#     replay_buffer.add_batch(batch)
#     time_step = next_time_step

In [11]:
# for _ in range(10):
#     experience, unused_info = next(iterator)
#     train_loss = agent.train(experience)
# agent_checkpointer.save(train_step_counter)

In [16]:
for _ in range(15):
    env.close()
    env = EnvTensorOne(reward_function = reward, turbo_mode=False, **args)
    time_step = env.reset(policy=agent.policy)
    while not time_step.is_last():
        action_step = agent.collect_policy.action(time_step)
        next_time_step = env.step(action_step.action)
        traj = trajectory.from_transition(time_step, action_step, next_time_step)
        batch = tf.nest.map_structure(lambda t: tf.expand_dims(t, 0), traj)
        replay_buffer.add_batch(batch)
        time_step = next_time_step
    for _ in range(15):
        experience, unused_info = next(iterator)
        train_loss = agent.train(experience)
        print(train_loss.loss.numpy())
    agent_checkpointer.save(train_step_counter)

controlled:  0
total_reward: 0.0
0.2381017
0.55774426
0.2145111


KeyboardInterrupt: 

In [ ]:
train_step_counter

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=8050>

In [ ]:
saver.save(f"ppo")

INFO:tensorflow:Assets written to: ppo/assets


/home/liliaceae/anaconda3/envs/tf/lib/python3.9/site-packages/tensorflow/python/saved_model/nested_structure_coder.py:497: UserWarning: Encoding a StructuredValue with type tfp.distributions.Deterministic_ACTTypeSpec; loading this StructuredValue will require that this type be imported and registered.
  warnings.warn("Encoding a StructuredValue with type %s; loading this "
INFO:tensorflow:Assets written to: ppo/assets


In [ ]:
# env.close()
# env = EnvTensorSix(reward_function = reward, turbo_mode=True, **args)
# time_step = env.reset(policy=agent.policy)
# action_step = agent.collect_policy.action(time_step)


In [ ]:
# # Reset the train step.
# agent.train_step_counter.assign(0)

# # Evaluate the agent's policy once before training.
# avg_return = compute_avg_return(env, agent.policy, 2)
# returns = [avg_return]

# # Reset the environment.
# time_step = env.reset()

# # Create a driver to collect experience.
# collect_driver = dynamic_step_driver.DynamicStepDriver(
#     env,
#     agent.collect_policy,
#     observers=None,
#     num_steps=32) 

In [ ]:
# class Agent:
#     def __init__(self, shape_of_model: tuple):
#         nb_outputs = 15
#         self.model = keras.models.Sequential()
#         self.model.add(layers.Input(shape=(64,)))
#         for n in shape_of_model:
#             self.model.add(layers.Dense(units=n, activation='relu'))
#         self.model.add(layers.Dense(units=nb_outputs, activation='linear'))
    
#     def action(self, state: np.ndarray):
#         prediction = self.model.predict(state)
#         # min max scaling
        
#         return self.model.predict(state)
    


In [ ]:
# class AgentManager:
#     def __init__(self, env, number_of_agents):
#         self.env = env
#         self.agents = [Agent(shape_of_model=(15,15)) for i in range(number_of_agents)]
            
#     def take_action(self, observation):
#         return [agent.action(observation) for agent in self.agents]   
        
        